# 01 — Placeholder data inspection

In [ ]:
# Ensure src/ is importable when running without an editable install.
import sys
from pathlib import Path

here = Path().resolve()
candidates = [here] + list(here.parents)
src_path = None
for c in candidates:
    if (c / "src").exists():
        src_path = c / "src"
        break
    if (c / ".." / "src").resolve().exists():
        src_path = (c / ".." / "src").resolve()
        break
if src_path and str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

In [ ]:
from pathlib import Path
import pandas as pd
from inkswarm_detectlab.io.paths import raw_table_basepath
from inkswarm_detectlab.io.tables import read_auto
from inkswarm_detectlab.utils.run_id import PLACEHOLDER_RUN_ID

run_dir = Path("runs") / "RUN_XXX_0005"  # set to your run id

login = read_auto(raw_table_basepath(run_dir, "login_attempt"))
checkout = read_auto(raw_table_basepath(run_dir, "checkout_attempt"))

print("login shape:", login.shape)
display(login.head())

print("checkout shape:", checkout.shape)
display(checkout.head())
